In [1]:
pip install --upgrade scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


In [3]:
import sys
sys.executable


'E:\\New folder\\python.exe'

In [4]:
!pip install scikit-learn


In [5]:
import pandas as pd
import numpy as np
import sklearn
import nltk
import flask

print("All packages imported successfully!")
print("Scikit-Learn Version:", sklearn.__version__)
print("NLTK Version:", nltk.__version__)


All packages imported successfully!
Scikit-Learn Version: 1.6.1
NLTK Version: 3.8.1


In [6]:
pip install pandas numpy scikit-learn nltk flask


In [7]:
pip install pandas numpy scikit-learn nltk flask


Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
import re
import string
import nltk

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

nltk.download("stopwords")
nltk.download("wordnet")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mehar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mehar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
import pandas as pd

# Load only 10,000 rows to reduce processing time
fake_data = pd.read_csv("Fake.csv", nrows=500)
true_data = pd.read_csv("True.csv", nrows=500)

print("Dataset loaded successfully!")

# Add labels: 1 for True, 0 for Fake
fake_data["label"] = 0
true_data["label"] = 1

# Combine datasets
df = pd.concat([fake_data, true_data])

# Shuffle the dataset to mix fake and real news
df = df.sample(frac=1).reset_index(drop=True)

print("Dataset prepared successfully!")


Dataset loaded successfully!
Dataset prepared successfully!


In [10]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\[.*?\]', '', text)  # Remove text in brackets
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>+', '', text)  # Remove HTML tags
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)  # Remove punctuation
    text = re.sub(r'\n', '', text)  # Remove newlines
    text = re.sub(r'\w*\d\w*', '', text)  # Remove words containing numbers
    
    # Tokenize and remove stopwords
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return " ".join(words)

# Apply text preprocessing
df["cleaned_text"] = df["text"].apply(clean_text)

# Show sample processed text
print(df[["text", "cleaned_text"]].head())


                                                text  \
0  Despite the fact that Donald Trump and his fan...   
1  WASHINGTON (Reuters) - The top Democrat on the...   
2  James Clapper, the Director of National Intell...   
3  (Reuters) - U.S. President Donald Trump has ag...   
4  You may not be familiar with the name  Unileve...   

                                        cleaned_text  
0  despite fact donald trump fanbase try convince...  
1  washington reuters top democrat senate judicia...  
2  james clapper director national intelligence p...  
3  reuters u president donald trump agreed meet r...  
4  may familiar name unilever good chance product...  


In [11]:
# Convert text into TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000)  # Limit to 5000 features for efficiency
X = vectorizer.fit_transform(df["cleaned_text"])
y = df["label"]

# Split data into training & testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# Train Naïve Bayes Model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Train Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [13]:
# Predict using Naïve Bayes
y_pred_nb = nb_model.predict(X_test)
print("Naïve Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

# Predict using Random Forest
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Naïve Bayes Accuracy: 0.965
              precision    recall  f1-score   support

           0       0.99      0.93      0.96        89
           1       0.95      0.99      0.97       111

    accuracy                           0.96       200
   macro avg       0.97      0.96      0.96       200
weighted avg       0.97      0.96      0.96       200

Random Forest Accuracy: 0.995
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        89
           1       1.00      0.99      1.00       111

    accuracy                           0.99       200
   macro avg       0.99      1.00      0.99       200
weighted avg       1.00      0.99      1.00       200



In [14]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mehar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [15]:
%%writefile app.py

from flask import Flask, render_template, request
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re

# Download required NLTK data
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")

# Load the trained model and vectorizer
with open("model.pkl", "rb") as model_file:
    model = pickle.load(model_file)

with open("vectorizer.pkl", "rb") as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)

app = Flask(__name__)

# Preprocessing function
def preprocess_text(text):
    if not text.strip():  # Handle empty input
        return ""

    text = re.sub(r"[^a-zA-Z]", " ", text)  # Remove special characters & numbers
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenization
    tokens = [word for word in tokens if word not in stopwords.words("english")]  # Remove stopwords
    
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    tokens = [stemmer.stem(word) for word in tokens]  # Stemming

    return " ".join(tokens)

@app.route("/", methods=["GET", "POST"])
def home():
    prediction = None
    result = None

    if request.method == "POST":
        news_text = request.form.get("news", "").strip()  # Get input & handle empty string
        
        if news_text:
            processed_text = preprocess_text(news_text)
            text_vector = vectorizer.transform([processed_text])  # Convert text to vector
            prediction = model.predict(text_vector)[0]  # Predict (0 = Fake, 1 = Real)
            result = "Real News ✅" if prediction == 1 else "Fake News ❌"
        else:
            result = "Please enter a news article!"

    return render_template("index.html", result=result, news_text=news_text if request.method == "POST" else "")

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app.py


In [16]:
import pickle

# Save models & vectorizer
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))
pickle.dump(nb_model, open("fake_news_model.pkl", "wb"))


In [17]:
%%writefile templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>Fake News Detector</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            text-align: center;
            margin: 50px;
        }
        textarea {
            width: 60%;
            height: 100px;
        }
        input {
            padding: 10px 20px;
            font-size: 16px;
            margin-top: 10px;
            cursor: pointer;
        }
        .result {
            margin-top: 20px;
            font-size: 20px;
            font-weight: bold;
            color: blue;
        }
    </style>
</head>
<body>
    <h1>Fake News Detection System</h1>
    <form method="post">
        <textarea name="news" placeholder="Enter news article text here...">{{ news_text }}</textarea>
        <br>
        <input type="submit" value="Check News">
    </form>
    
    {% if result is not none %}
    <div class="result">Prediction: {{ result }}</div>
    {% endif %}
</body>
</html>


Overwriting templates/index.html


In [18]:
import pandas as pd
import numpy as np
import re
import nltk
import pickle

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Download necessary NLTK data
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")

# ✅ Load the datasets (Fixed variable names)
fake_df = pd.read_csv("Fake.csv", nrows=500)  
true_df = pd.read_csv("True.csv", nrows=500)  

print("✅ Dataset loaded successfully!")

# ✅ Add labels
fake_df["label"] = 0  # Fake news
true_df["label"] = 1  # True news

# ✅ Combine datasets
df = pd.concat([fake_df, true_df], axis=0)
df = df.sample(frac=1).reset_index(drop=True)  # Shuffle dataset

# ✅ Check if 'text' column exists
if "text" not in df.columns:
    raise ValueError("❌ Column 'text' not found in dataset!")

# ✅ Print sample text before preprocessing
print("📌 Sample text before processing:", df["text"].head(3).tolist())

# 🔹 Preprocessing function
def preprocess_text(text):
    text = re.sub(r"[^a-zA-Z]", " ", text)  # Remove special characters & numbers
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenization
    tokens = [word for word in tokens if word not in stopwords.words("english")]  # Remove stopwords
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    return " ".join(tokens)

# ✅ Debugging: Print status
print("🚀 Starting preprocessing...")  

# ✅ Apply preprocessing (Fixed multiprocessing issue)
df["processed_text"] = df["text"].apply(preprocess_text)

print("✅ Preprocessing complete!")

# ✅ Splitting dataset
print("🔹 Splitting dataset...")  
X_train, X_test, y_train, y_test = train_test_split(df["processed_text"], df["label"], test_size=0.2, random_state=42)

# ✅ Convert text into TF-IDF vectors
print("🔹 Vectorizing text...")  
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# ✅ Train the Naïve Bayes model
print("🔹 Training the model...")  
trained_model = MultinomialNB()
trained_model.fit(X_train_tfidf, y_train)

print("✅ Model training complete!")  

# ✅ Evaluate the model
print("🔹 Evaluating the model...")  
y_pred = trained_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)

print(f"🎯 Model Accuracy: {accuracy:.2f}")  # Should print accuracy ~90%

# 🔹 Save the trained model and vectorizer
with open("model.pkl", "wb") as file:
    pickle.dump(trained_model, file)

with open("vectorizer.pkl", "wb") as file:
    pickle.dump(vectorizer, file)

print("✅ Model and Vectorizer Saved Successfully!")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mehar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mehar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mehar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✅ Dataset loaded successfully!
📌 Sample text before processing: ['WASHINGTON (Reuters) - Nikki Haley, the U.S. ambassador to the United Nations, said on Sunday that any woman who has felt violated or mistreated by a man has every right to speak up, even if she is accusing President Donald Trump. “Women who accuse anyone should be heard,” Haley said on CBS’s “Face the Nation.” “They should be heard, and they should be dealt with.” Washington has been roiled by sexual misconduct scandals, with accusations leading to the resignations last week of three members of Congress. The growing wave of women reporting abuse or misconduct has brought down powerful men, from movie producer Harvey Weinstein to popular television personality Matt Lauer. Haley, discussing that cultural shift, applauded the women who have come forward: “I’m proud of their strength. I’m proud of their courage.” Asked how people should assess the accusers of the president, Haley said, it was “the same thing.” More than 10 

In [19]:
import os
print(os.getcwd())  # Ensure it's inside print()
os.getcwd()  # Display the path


C:\Users\mehar


'C:\\Users\\mehar'

In [20]:
import pickle

# Ensure 'trained_model' and 'vectorizer' are properly trained
with open("model.pkl", "wb") as file:
    pickle.dump(trained_model, file)

with open("vectorizer.pkl", "wb") as file:
    pickle.dump(vectorizer, file)

print("Model saved successfully!")


Model saved successfully!


In [21]:
import pickle

# Load the trained model
with open("model.pkl", "rb") as model_file:
    model = pickle.load(model_file)

# Check the model type
print("Model type:", type(model))


Model type: <class 'sklearn.naive_bayes.MultinomialNB'>


In [22]:
#Run On Command Prompt BY python app.py on Command prompt